# Historical FED Policy Rates vs Expectations
© 2026 Jim Domeij & Marek Ozana

This chart reconstructs how expectations for the Fed funds rate evolved over time. Each line is a year-end snapshot (`AS_OF_DATE`) of Bloomberg’s median economist path for future policy rates, shown alongside realized settings. Solid segments indicate historical (actual) policy rates, while dashed segments show the forecast that was in place on that snapshot date.


In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/macro") / "fed-history-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.Chart(...)

In [2]:
# Get historical and forecast policy rates for FED and other central banks
import polars as pl

from polars_bloomberg import BQuery

with BQuery() as bq:
    lst = []
    for year in range(2014, 2026):
        df = bq.bql(
            f"""
            let(#policy_rate = policy_rate(pr=range(2014Q4, 2027Q2), pt=Q, act_est_data=AE);)
            get(#policy_rate)
            for(['US'])
            with(dates='{year:d}-12-31')
            preferences(
                dropCols=[
                    'TICKER',
                    'REVISION_DATE',
                    'RATE_TYPE',
                    'REPORTED_DATE',
                    'CONCEPT',
                    'ECO_SOURCE',
                    'ACT_EST_SOURCE',
                    'PERIOD',
                    'PERIOD_END_DATE',
                    'SEASONALITY'
                ]
            )
            """
        ).combine()
        lst.append(df)

df_tot = pl.concat(lst)
df_tot.head()

ID,#policy_rate,PERIOD_REFERENCE_DATE,AS_OF_DATE,ACT_EST_DATA
str,f64,date,date,str
"""US""",0.25,2014-12-31,2014-12-31,"""A"""
"""US""",0.25,2015-03-31,2014-12-31,"""E"""
"""US""",0.4,2015-06-30,2014-12-31,"""E"""
"""US""",0.650001,2015-09-30,2014-12-31,"""E"""
"""US""",0.950001,2015-12-31,2014-12-31,"""E"""


In [3]:
# Create chart

import altair as alt

chart = (
    alt.Chart(
        df_tot.with_columns(pl.col("#policy_rate").mul(0.01).alias("pr")),
        title="FED Policy Rate vs Expectations",
    )
    .mark_line()
    .encode(
        x=alt.X("PERIOD_REFERENCE_DATE:T").title(None),
        y=alt.Y("pr:Q", title="Policy Rate (%)").axis(format="%"),
        detail=alt.Detail("AS_OF_DATE:N"),
        color=alt.Color("ACT_EST_DATA:N").title("Actual/Estimate"),
        size=alt.condition(alt.datum.ACT_EST_DATA == "A", alt.value(3), alt.value(1.5)),
        strokeDash=alt.StrokeDash("ACT_EST_DATA:N").title("Actual/Estimate"),
        opacity=alt.condition(
            alt.datum.ACT_EST_DATA == "A", alt.value(0.5), alt.value(1)
        ),
        tooltip=[
            "ID:N",
            "PERIOD_REFERENCE_DATE:T",
            "AS_OF_DATE",
            "#policy_rate:Q",
            "ACT_EST_DATA:N",
        ],
    )
    .properties(width=500, height=400)
)

# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)